<a href="https://colab.research.google.com/github/truthbutcher/studymaterials/blob/main/smallpath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
data = pd.read_csv('paths.csv')
d = pd.read_csv('fulldata.csv')
data = data.iloc[:,:10]
data.iloc[:,0] = data.iloc[:,0].astype(str)

posdict = {}
for index, linkid in enumerate(d.iloc[:,0]):
  posdict[int(linkid)] = d.iloc[index,6]

for index, linkid in enumerate(data.iloc[:,0]):
  data.iloc[index,6] = posdict[int(linkid)]

print(data.head(20)) 

        linkid  link轨迹数量        平均速度  ...  link平均位置          宽度  widthmark
0   1620600236         3  296.333333  ...       115      无效-已移除        NaN
1   1584789428         1  256.000000  ...       100    2.4-2.7M        0.0
2   1615902250         4   33.750000  ...       107    2.4-2.7M        0.0
3   1632660439      1027   17.136319  ...        62      无效-已移除        NaN
4   1634218424       350   16.288571  ...        95      无效-已移除        NaN
5   1601397551         2   14.000000  ...        44       >2.7M        0.0
6   1632495868         1   14.000000  ...       161       >2.7M        0.0
7   1586589883        10   13.800000  ...        25   1.8-2.4 M        1.0
8   1582842109        29   12.586207  ...        23    2.4-2.7M        0.0
9   1604510220        10   12.400000  ...         9      无效-已移除        NaN
10  1621154479         3   11.666667  ...        30    2.4-2.7M        0.0
11  1636345982         3   11.666667  ...        32   1.8-2.4 M        1.0
12  1653112438         2 

In [ ]:
data_full = data.copy()
data_full = data_full.dropna(axis = 0)
X_data = data_full.drop(['linkid','宽度','widthmark'], axis = 1)
X_data['link轨迹数量'] = pd.to_numeric(X_data['link轨迹数量'])
X_data['平均速度'] = pd.to_numeric(X_data['平均速度'])
X_data['link平均位置'] = pd.to_numeric(X_data['link平均位置'])
#X_data['4'] = pd.to_numeric(X_data['<4m/s 的轨迹'])
#X_data['4-8'] = pd.to_numeric(X_data['4 - 8m/s 的轨迹'])
#X_data['8'] = pd.to_numeric(X_data['>8m/s 的轨迹'])
X_data = X_data.drop(['>8m/s 的轨迹','<4m/s 的轨迹','4 - 8m/s 的轨迹'],axis = 1)
y = data_full.widthmark
X_data.head()

,link轨迹数量,平均速度,link平均位置
1,1,256.00,100
2,4,33.75,107
5,2,14.00,44
6,1,14.00,161
7,10,13.80,25


In [ ]:
seed = 7
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=test_size, random_state=seed)
kf = KFold(n_splits=5)
X_train

,link轨迹数量,平均速度,link平均位置
114,11,3.636364,7
147,3,1.333333,75
20,2,9.500000,13
126,2,3.000000,3
70,11,5.454545,34
...,...,...,...
227,2,4.000000,3
79,1,5.000000,1
32,19,8.052632,12
212,1,2.000000,10


In [ ]:
model1 = xgb.XGBClassifier()
model2 = xgb.XGBClassifier(
 learning_rate =0.01,
 n_estimators=100,
 max_depth=4,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,  
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
train_model1 = model1.fit(X_train, y_train)
train_model2 = model2.fit(X_train, y_train)

In [ ]:
pred1 = train_model1.predict(X_test)
pred2 = train_model2.predict(X_test)

print(classification_report(y_test, pred1))
print(classification_report(y_test, pred2))

print("Accuracy for model 1: %.2f" % (accuracy_score(y_test, pred1) * 100))
print("Accuracy for model 2: %.2f" % (accuracy_score(y_test, pred2) * 100))


              precision    recall  f1-score   support

         0.0       0.60      0.76      0.67        41
         1.0       0.47      0.30      0.37        30

    accuracy                           0.56        71
   macro avg       0.53      0.53      0.52        71
weighted avg       0.54      0.56      0.54        71

              precision    recall  f1-score   support

         0.0       0.60      0.78      0.68        41
         1.0       0.50      0.30      0.37        30

    accuracy                           0.58        71
   macro avg       0.55      0.54      0.53        71
weighted avg       0.56      0.58      0.55        71

Accuracy for model 1: 56.34
Accuracy for model 2: 57.75


In [ ]:
output = pd.read_csv('路障数据.csv')
output.head()

,linkid,link轨迹数量,平均速度,link平均位置,4,4-8,8,widthmark
0,1526653217,34,6.029412,8,4,21,9,NaN
1,1582531367,31,5.806452,8,4,19,8,NaN
2,1529054785,61,8.459016,55,2,31,28,NaN
3,1528904608,61,4.852459,53,14,44,3,NaN
4,1530715556,36,6.472222,42,1,23,12,NaN


In [ ]:
output_data = output.drop(['linkid','widthmark'], axis = 1)
output_data['link轨迹数量'] = pd.to_numeric(output_data['link轨迹数量'])
output_data['平均速度'] = pd.to_numeric(output_data['平均速度'])
output_data['link平均位置'] = pd.to_numeric(output_data['link平均位置'])
output_data['4'] = pd.to_numeric(output_data['4'])
output_data['4-8'] = pd.to_numeric(output_data['4-8'])
output_data['8'] = pd.to_numeric(output_data['8'])


output_data.head()



,link轨迹数量,平均速度,link平均位置,4,4-8,8
0,34,6.029412,8,4,21,9
1,31,5.806452,8,4,19,8
2,61,8.459016,55,2,31,28
3,61,4.852459,53,14,44,3
4,36,6.472222,42,1,23,12


In [ ]:
output_pred = train_model2.predict(output_data)
output['widthmark'] = pd.to_numeric(output_pred)


,linkid,link轨迹数量,平均速度,link平均位置,4,4-8,8,widthmark
0,1526653217,34,6.029412,8,4,21,9,1.0
1,1582531367,31,5.806452,8,4,19,8,1.0
2,1529054785,61,8.459016,55,2,31,28,0.0
3,1528904608,61,4.852459,53,14,44,3,0.0
4,1530715556,36,6.472222,42,1,23,12,0.0


In [ ]:
output.to_excel('路障数据1.xlsx', sheet_name = '路障数据')